# Lab 2 - Regression, DiD, and Fixed Effects
- **Author:** Suraj R. Nair ([suraj.nair@berkeley.edu](mailto:suraj.nair@berkeley.edu)) (based on past labs by Emily Aiken, Qutub Khan Vajihi and Dimitris Papadimitriou)
- **Date:** February 24, 2024
- **Course:** INFO 251: Applied machine learning

### Topics:
1. Linear/ Multivariate Regression
2. Dummy variables
3. Exercises

### References: 
 * [Statsmodels](http://www.statsmodels.org/stable/example_formulas.html#loading-modules-and-functions) 
 * [Interpreting regression coefficients](https://dss.princeton.edu/online_help/analysis/interpreting_regression.htm)
 * [Card and Krueger (1994)](https://davidcard.berkeley.edu/papers/njmin-aer.pdf)
 * Fixed Effects:
      - [Panel Data and Fixed Effects in Python](https://matheusfacure.github.io/python-causality-handbook/14-Panel-Data-and-Fixed-Effects.html)
      - [Notes](https://www.jblumenstock.com/files/courses/econ174/FEModels.pdf) on fixed effects

### Import libraries

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline  

# The packages you'll need for regression models
import statsmodels.api as sm
import statsmodels.formula.api as smf

### 1. Load the data

Card and Krueger (1994) collected survey data on employment in fast food restaurants in New Jersey and Pennsylvania in 1992. The data for today's lab uses a subset of the variables they collected.

- *UNIQUE_ID*: Unique ID for the restaurant interviewed
- *PERIOD*: 0 for pre-period (March 1994), 1 for post-period (December 1994)
- *STATE*: 0 for Pennsylvania, 1 for New Jersey
- *REGION*: Region code: 1 = Southern NJ, 2 = Central NJ, 3 = Northern NJ, 4 = Northeast Philly suburbs, 5 = Easton area, 6 = NJ Shore
- *CHAIN*: Chain restaurant code: 1 = Burger King, 2 = KFC, 3 = Roy's, 4 = Wendy's
- *EMP*: Number of employees (fulltime or parttime)
- *CO_OWNED*: 1 if company-owned
- *BONUS*: 1 if employees get a signing bonus
- *HRSOPEN*: Hours open per day, up to 24
- *NREGS*: Number of registers

In [ ]:
df = pd.read_csv('fastfood.csv')
df.head()

In [ ]:
df.tail()

In [ ]:
#Create a variable identifying the pre-treatment period
pre = df[df['PERIOD'] == 0].copy()
post = df[df['PERIOD'] == 1].copy()

### 2. Univariate Regression

Linear regression provides us a concise summary of one variable as a function of another variables(s) through two types of parameters - the slope and the intercept. To review linear regression, we'll start by exploring the relationship between employment and the number of hours a restaurant is open
in the pre-period.

#### 2.1 Exploratory analysis

In [ ]:
# Find the correlation between number of rooms and median housing price.
np.corrcoef(pre['HRSOPEN'], pre['EMP'])[0][1]

#Pandas also has a built correlation function
# pre[['HRSOPEN', 'EMP']].corr()

In [ ]:
fig = plt.figure(figsize=(8, 5))
plt.scatter(pre['HRSOPEN'], pre['EMP'], alpha=.2)
plt.xlabel('Daily Hours Open', fontsize='large')
plt.ylabel('Number of Employees', fontsize='large')
plt.title('Hours Open vs. Employees', fontsize='x-large')
plt.xlim(6.9, 24.1)
sns.despine()
plt.show()

#### 2.2 Estimating a regression with np.polyfit

In [ ]:
# Estimate the regression
x, y = pre['HRSOPEN'].values, pre['EMP'].values # x is the input variable, y is the output variable
slope, intercept = np.polyfit(x, y, 1) # 1 is the degree

# Scatterplot with the regression line
fig = plt.figure(figsize=(8, 5))
plt.scatter(pre['HRSOPEN'], pre['EMP'], alpha=.2)
plt.plot(x, slope*x + intercept, color='darkgrey')
plt.xlabel('Daily Hours Open', fontsize='large')
plt.ylabel('Number of Employees', fontsize='large')
plt.title('Hours Open vs. Employees', fontsize='x-large')
plt.xlim(6.9, 24.1)
sns.despine()
plt.show()

#### 2.3 Interpretation of the slope and intercept

In [ ]:
print(f'The slope of the line is {np.round(slope, 3)}')

**Question**: How would you interpret this value?

In [ ]:
print(f'The intercept of the line is {np.round(intercept, 3)}')

**Question**: How would you interpret this value?

#### 2.4 Estimating a regression with statsmodels

In [ ]:
# Syntax option 1
x, y = pre['HRSOPEN'].values, pre['EMP'].values # x is the input variable, y is the output variable
x = sm.add_constant(x) # Add a constant for the intercept term
model1 = sm.OLS(y, x).fit() # Note the order of y folowed by x!
print(model1.summary())

In [ ]:
# Syntax option 2
model2 = smf.ols(formula='EMP ~ HRSOPEN', data=pre).fit() # Automatically includes the intercept term
print(model2.summary())

### 2. Categorical Data

Now, we'll experiment with categorical data by examining the relationship between EMPTOT (the number of employees) and CHAIN (the fast food chain category) in the pre-period.

In [ ]:
# Check values of CHAIN
pre['CHAIN'].value_counts()

In [ ]:
# Get dummy variables for CHAIN
dummy_pre = pd.get_dummies(pre, columns=['CHAIN']).head() # Pandas' default is not to drop a column
dummy_pre.head()

In [ ]:
# Get dummy variables for CHAIN and drop one column
dummy_pre = pd.get_dummies(pre, columns=['CHAIN'], drop_first=True).head() # Drop a column
dummy_pre.head()

In [ ]:
# Regression with a dummy variable: Syntax option 1
x = pd.get_dummies(pre[['CHAIN']], columns=['CHAIN'], drop_first=True)
x = sm.add_constant(x)
y = pre['EMP']
print(sm.OLS(y, x).fit().summary())

**Exercise**: 

- How do we interpret the coefficients on the variable(s) CHAIN?

In [ ]:
# Regression with a dummy variable: Syntax option 2

# Statsmodels formula API automatically drops one of the dummies
print(smf.ols(formula='EMP ~ C(CHAIN)', data=pre).fit().summary()) 

 - Now run the regression again, this time dropping the constant. What do you observe? How have the coefficients changed?

In [ ]:
# Alternative: drop the constant
print(smf.ols(formula='EMP ~ C(CHAIN) - 1', data=pre).fit().summary()) 

### 3. Multivariate Regression

Now let's look at some other covariates: The number of registrations, whether or not employees get a bonus, and the region.

In [ ]:
# Syntax 1
X = sm.add_constant(pre[['HRSOPEN', 'NREGS', 'BONUS']]) # X is capitalized since it's now a matrix
y = pre['EMP']
model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
# Syntax 2
model = smf.ols(formula='EMP ~ HRSOPEN + NREGS + C(REGION)', data=pre).fit()
print(model.summary())

**QUESTION**: Interpret each of the regression coefficients.

# Exercises:


1. On average, prior to the wage increase, is the average employment in New Jersy and Pennsylvania similar? 

2. Calculate the average change in employment, for the following groups (difference in means):
    
    - Pennsylvania v/s New Jersey (Post-period only, i.e PERIOD = 1)
    - Pennsylvania v/s New Jersey (Pre-period only, i.e PERIOD = 0)
    - Pre v/s Post in Pennsylvania
    - Pre v/s Post in New Jersey 
    - Differences-in-difference

3. Now, let's do the same thing, but using linear regression! Let's focus on differences between Pennsylvania and New Jersey in the pre-period only. What do you observe?



4. Let's see what happens when we add a control variable. Let's control for the number of hours (HRSOPEN) 

5. Plot the regression line/ line of best fit in the treatment and the control groups

6. Let's finish with a visual assessment of trends in EMP before and after the wage increase, in T and C

In [ ]:

# Let's calculate the mean for the pre and post periods, in both the treatment and control groups. 
control_pre = 
treatment_pre = 
control_post = 
treatment_post = 


# Now, let's plot this. 
fig = plt.figure(figsize=(8, 5))
plt.scatter([0, 1], [control_pre, control_post], s=200)
plt.plot([0, 1], [control_pre, control_post], label='Control (Pennsylvania)')
plt.scatter([0, 1], [treatment_pre, treatment_post], s=200)
plt.plot([0, 1], [treatment_pre, treatment_post], label='Treatment (New Jersey)')
plt.legend(loc='best', fontsize='large')
plt.xlabel('Time (Pre vs. Post)', fontsize='large')
plt.ylabel('Average Employees', fontsize='large')
plt.title('Employees Over Time', fontsize='x-large')
plt.show()


## FOR NEXT WEEK

### Differences-in-differences


Use a differences-in-differences regression specification to estimate the impact of the increase in the minimum wage in New Jersey (state == 1) between the pre- and the post-period (period==1). 

Remember the diff-in-diff regression specification: Y = B0 + B1 * Time + B2 * Intervention + B3*(Time*Intervention)

**QUESTION**: Interpret the regression coefficients in terms of the impact of the minimum wage change on employment.

**QUESTION**: Why might you want to add control variables to this regression?

### 5. Interaction Terms

An interaction term captures the notion that the value of a variable is influenced by the value of another variables. 

In [ ]:
model1 = smf.ols(formula='EMP ~ HRSOPEN + BONUS + HRSOPEN * BONUS ', data=pre).fit()
print(model1.summary())

**QUESTION**: Interpret the coefficients

**QUESTION**: Create a plot which demonstrates how the variable BONUS influences the value of HRSOPEN

### 6: Fixed Effects Intuition

Let's create a toy dataset for this example, with these variables:

    - city: New York, Chicago, San Francisco, Washington D.C
    - year: 2020, 2021, 2022, 2023, 2024
    - quantity: the amount purchased (of some arbitrary good)
    - price: the cost of the arbitrary good, varying by year and city. 
   

In [ ]:

df_panel = pd.DataFrame({
    "quantity": [25, 24.3, 23, 20,
                 15, 13.8, 12.5, 13,
                 20, 19, 18, 19,
                 22, 19.5, 19, 19,
                ],
    "price": [5.0, 5.34, 5.47, 5.51, 
              4.01, 4.35, 4.43, 4.43, 
              4.50, 4.62, 4.70, 4.89, 
             4.80, 4.90, 4.95, 5.25
             ],
    "year":[2020, 2021, 2022, 2023]*4,
    "city":  ['New York']*4 +  
             ["Chicago"]*4 + 
             ["San Francisco"]*4 + 
             ["Washington D.C."]*4
})


Let's explore a cross-section of this data, say in the year 2022.  

In [ ]:
### View the data

df_panel[df_panel['year'] == 2022]

Simply looking over this data, it actually appears the quantity purchased appears to increase along with prices. Does that make sense?. 

What might explain this? On the one hand, perhaps the quality of the good is higher in certain cities. For example, if the **good** in question is a bagel, perhaps the best bagels are found in New York (ahem, Boichik)? Alternatively, there might simply be a high demand for bagels in New York, thus explaining the high prices. 

But simply put, we can't say much about the role of price here -- since we do not have data for quality, or bagel obsession.. Running a regression to further examine the relationship between quantity and price is likely to thus suffer from omitted variable bias.

To hammer home this point, let's explore the relationship between quantity and price using a univariate regression, across the entire sample. What do you get?

In [ ]:
m = smf.ols("quantity ~ price", data = df_panel).fit()

sns.scatterplot(data = df_panel, x=  'price', y = 'quantity', hue = "city")
plt.plot(df_panel['price'], m.fittedvalues, "k-", alpha = 0.5, label="Regression Line")
plt.xlabel("Price")
plt.ylabel("Quantity")
sns.despine()
plt.show()

Now, let's examine the full dataset. 

In [ ]:
df_panel

We see that as price increases, the quantity purchased decreases **within** each city. This is reasonable, as we would expect the demand to fall as price increases.

This provides the underlying intuition for fixed effects. In the cross-section we saw earlier, a regression that primarily relied on variation across cities is problematic, due to omitted variables bias. The solution is to focus on **within** city variation, relying on the panel structure of the data to remove the effect of omitted variable bias.

We still have to make a key assumption here, however. What is it?

Now, let's examine a simple fixed effects model, where we eliminate the unobserved variation across cities, and focus only on within city variation. A simple way to achieve this, as you saw in lecture, is to add a dummy variable for each city. So let's go ahead and try that below. 

In [ ]:
#C() is used to tell statsmodels that the variable in question is a categorical/ factor variable
fe = smf.ols("quantity ~ price + C(city)", data=df_panel).fit()  

df_panel['y_hat'] = fe.fittedvalues

sns.scatterplot(data = df_panel, x = 'price', y = 'quantity', hue = 'city')
for city in df_panel["city"].unique():
    plt_df = df_panel[df_panel["city"] == city]
    plt.plot(plt_df['price'], plt_df['y_hat'], "k-", alpha = 0.5)

plt.title("Fixed Effect Model")
plt.xlabel("Quantity")
plt.ylabel("Price")
plt.legend(loc = "best")
sns.despine()
plt.show()

Notice that we are effectively fitting one regression line per city. This is what the ''city'' fixed effect does -- it controls for the average (time invariant) differences across cities (in observable/ unobservable variables). 